In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# 데이터 로드
df = pd.read_csv('시군구별전력사용량.csv', encoding='cp949')  # encoding='cp949' 추가

# 필요한 열 선택 및 조건 필터링
df = df[['시도', '시군구', '계약종별', '1월', '2월', '3월', '4월', '5월']]
df = df[df['계약종별'] == '합 계']

# 결과 저장할 리스트
results = []

for index, row in df.iterrows():
    # 1월부터 4월까지 데이터를 사용하여 5월 예측
    X = [[1], [2], [3], [4]]  # 월
    y = [row['1월'], row['2월'], row['3월'], row['4월']]
    model = LinearRegression()
    model.fit(X, y)

    # 5월 예측
    predicted_may = model.predict([[5]])[0]

    # 실제 5월 값
    actual_may = row['5월']

    # 결과 저장
    results.append({
        '시도': row['시도'],
        '시군구': row['시군구'],
        '예측값': np.floor(predicted_may),
        '실제값': actual_may,
        '오차': np.floor(abs(predicted_may - actual_may))
    })

# 결과를 DataFrame으로 변환
result_df = pd.DataFrame(results)

# 결과 출력 또는 저장
result_df
# result_df.to_csv('prediction_results.csv', index=False)


import pandas as pd
import matplotlib.pyplot as plt

# 한글 폰트 설정
plt.rcParams['font.family'] = 'NanumGothic'
# plt.rcParams['font.family'] = 'AppleGothic'  # MacOS
# plt.rcParams['font.family'] = 'NanumGothic'  # Ubuntu

# Calculate the absolute error and mean squared error for each province
result_df['절대오차'] = abs(result_df['예측값'] - result_df['실제값'])
result_df['제곱오차'] = (result_df['예측값'] - result_df['실제값']) ** 2

# MSE 계산
mse_by_province = result_df.groupby('시도')['제곱오차'].mean()
mae_by_province = result_df.groupby('시도')['절대오차'].mean()

# 그래프 시각화
plt.figure(figsize=(12, 6))

# MSE 그래프
plt.subplot(1, 2, 1)  # 1행 2열의 첫 번째 서브플롯
mse_by_province.plot(kind='bar', color='orange', alpha=0.7)
plt.xlabel('시도')
plt.ylabel('평균 제곱 오차 MSE')
plt.title('시도별 평균 제곱 오차 MSE')
plt.xticks(rotation=45, ha='right')

# MAE 그래프
plt.subplot(1, 2, 2)  # 1행 2열의 두 번째 서브플롯
mae_by_province.plot(kind='bar', color='skyblue', alpha=0.7)
plt.xlabel('시도')
plt.ylabel('평균 절대 오차 MAE')
plt.title('시도별 평균 절대 오차 MAE')
plt.xticks(rotation=45, ha='right')

plt.tight_layout()  # Adjust layout to prevent labels from overlapping
plt.show()
